In [2]:
%pwd

'c:\\Users\\Soumyadipta\\Desktop\\Exoplanet-Detection\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Soumyadipta\\Desktop\\Exoplanet-Detection'

In [5]:
from dataclasses  import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from exoplanetDetection.constants import *
from exoplanetDetection.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from exoplanetDetection import logger
from exoplanetDetection.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info : \n{headers}")
        
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function retrns None
        """   
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-15 10:42:57,747: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-15 10:42:57,752: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-15 10:42:57,754: INFO: common: created directory at: artifacts]
[2024-03-15 10:42:57,757: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-15 10:43:06,619: INFO: 366118637: artifacts/data_ingestion/data.zip downloaded with the following info : 
Connection: close
Content-Length: 19067749
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "37fc1675d0e0274bfb474ae34af63b1f3d5f1f7d00af3e2381eb68134199f611"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8112:12C163:A9C6:CFBC:65F3D8DB
Accept-Ranges: bytes
Date: Fri, 15 Mar 2024 05:13:01 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10221-MAA
X-Cache: